In [64]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [65]:
gdf = gpd.read_file("cb_2024_01_place_500k.shp")
gdf.head()

,STATEFP,PLACEFP,PLACENS,GEOIDFQ,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,01,32704,02403782,1600000US0132704,0132704,Haleyville,Haleyville city,AL,Alabama,25,21713018,130983,"MULTIPOLYGON (((-87.60417 34.24614, -87.60201 ..."
1,01,45472,02406131,1600000US0145472,0145472,McIntosh,McIntosh town,AL,Alabama,43,2579767,0,"POLYGON ((-88.03726 31.27171, -88.03112 31.271..."
2,01,53784,02406982,1600000US0153784,0153784,Newbern,Newbern town,AL,Alabama,43,3010370,6872,"POLYGON ((-87.54125 32.60104, -87.52531 32.601..."
3,01,78204,02405634,1600000US0178204,0178204,Valley,Valley city,AL,Alabama,25,33384157,0,"MULTIPOLYGON (((-85.17006 32.7799, -85.17005 3..."
4,01,02836,02405172,1600000US0102836,0102836,Ashford,Ashford city,AL,Alabama,25,16064130,56699,"POLYGON ((-85.25721 31.17963, -85.25643 31.204..."


In [66]:
gdf.shape, gdf.crs, list(gdf.columns)


((594, 13),
 <Geographic 2D CRS: EPSG:4269>
 Name: NAD83
 Axis Info [ellipsoidal]:
 - Lat[north]: Geodetic latitude (degree)
 - Lon[east]: Geodetic longitude (degree)
 Area of Use:
 - name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. Bri

In [67]:
gdf_m = gdf.to_crs(epsg=5070)  # NAD83 / Conus Albers (meters)

gdf_m.crs


<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [68]:
gdf_m["area_m2"] = gdf_m.geometry.area
gdf_m["perim_m"] = gdf_m.geometry.length

gdf_m[["area_m2", "perim_m"]].describe()



,area_m2,perim_m
count,5.940000e+02,594.000000
mean,2.774096e+07,48435.867623
std,4.595060e+07,70523.046388
min,2.818985e+05,2201.152889
25%,6.875652e+06,14829.446235
50%,1.505789e+07,27066.593675
75%,3.164486e+07,48554.370600
max,5.877682e+08,884004.149449


In [69]:
import numpy as np

gdf_m["circularity"] = (4 * np.pi * gdf_m["area_m2"]) / (gdf_m["perim_m"] ** 2)

gdf_m["circularity"].describe()
gdf_m[gdf_m["circularity"] >= 0.90]



,STATEFP,PLACEFP,PLACENS,GEOIDFQ,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry,area_m2,perim_m,circularity
256,01,55992,02407028,1600000US0155992,0155992,Oakman,Oakman town,AL,Alabama,43,7981750,22558,"POLYGON ((789804.377 1218833.469, 789846.897 1...",7.956614e+06,10037.473572,0.992406
289,01,30448,02406586,1600000US0130448,0130448,Goldville,Goldville town,AL,Alabama,43,2552525,39434,"POLYGON ((945110.668 1163400.975, 945129.601 1...",2.550451e+06,5705.138076,0.984679
383,01,16240,02406288,1600000US0116240,0116240,Coffee Springs,Coffee Springs town,AL,Alabama,43,2030176,29256,"POLYGON ((956097.865 949916.711, 956175.086 95...",2.002238e+06,5069.157018,0.979161
386,01,09016,02405315,1600000US0109016,0109016,Brantley,Brantley town,AL,Alabama,43,8047471,130826,"POLYGON ((917794.94 992407.281, 917860.403 992...",8.115637e+06,10131.186745,0.993601


In [70]:
top30 = gdf_m.sort_values("circularity", ascending=False)[
    ["NAME", "circularity", "area_m2"]
].head(30)

top30


,NAME,circularity,area_m2
386,Brantley,0.993601,8.115637e+06
256,Oakman,0.992406,7.956614e+06
289,Goldville,0.984679,2.550451e+06
383,Coffee Springs,0.979161,2.002238e+06
170,Mooresville,0.794309,7.813877e+05
68,Beaverton,0.789951,1.189502e+07
4,Ashford,0.785922,1.611041e+07
125,Carrollton,0.785647,5.389578e+06
523,Slocomb,0.785331,2.456535e+07
47,Daviston,0.785107,2.376369e+07


In [71]:
circular_towns = (
    gdf_m[gdf_m["circularity"] >= 0.90]
    [["NAME", "circularity", "area_m2"]]
    .sort_values("circularity", ascending=False)
)

circular_towns


,NAME,circularity,area_m2
386,Brantley,0.993601,8.115637e+06
256,Oakman,0.992406,7.956614e+06
289,Goldville,0.984679,2.550451e+06
383,Coffee Springs,0.979161,2.002238e+06
